<a href="https://colab.research.google.com/github/DamianoGiani/EGVSR/blob/master/VisualAndMultimedia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import

In [ ]:
import torch
from matplotlib import pyplot as plt
import numpy as np
import cv2
import time
import os, sys
from PIL import Image
import pickle
import torch.nn as nn
import pickle
import torch.nn.functional as F
import functools
import glob
from skimage.metrics import structural_similarity as ssim
import time


create folders to store results

In [ ]:
!mkdir -p /content/dataset/foreground/{GT,Gaussian}/foreground
!mkdir -p /content/dataset/background/{GT,Gaussian}/background
!mkdir -p /content/dataset/full/{GT,Gaussian}/full
!mkdir video
!mkdir -p results/{MyFirstMod,MySecondMod}/{EGVSR_iter420000,MyG_iter12000}

In [ ]:
#import shutil
#%cd /content
#shutil.rmtree('/content/results/MyThirdMod/EGVSR_iter420000')

import the video that will be processed

In [ ]:
!wget -nc https://github.com/DamianoGiani/VisualAndMultimediaRecognition/blob/main/video/footballMatchTrim.mp4?raw=true -O video/footballMatch.mp4 #3840*2160 frame dimension
#!wget -nc https://github.com/DamianoGiani/VisualAndMultimediaRecognition/blob/main/video/calcio_Trim.mp4?raw=true -O video/footballMatch.mp4 #1280*720 frame dimension

--2022-09-02 15:08:46--  https://github.com/DamianoGiani/VisualAndMultimediaRecognition/blob/main/video/footballMatchTrim.mp4?raw=true
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/DamianoGiani/VisualAndMultimediaRecognition/raw/main/video/footballMatchTrim.mp4 [following]
--2022-09-02 15:08:46--  https://github.com/DamianoGiani/VisualAndMultimediaRecognition/raw/main/video/footballMatchTrim.mp4
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/DamianoGiani/VisualAndMultimediaRecognition/main/video/footballMatchTrim.mp4 [following]
--2022-09-02 15:08:46--  https://raw.githubusercontent.com/DamianoGiani/VisualAndMultimediaRecognition/main/video/footballMatchTrim.mp4
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108

#Separate Background and Foreground using yolov5

load YOLO model

In [ ]:
yolo = torch.hub.load('ultralytics/yolov5', 'yolov5s',)

/usr/local/lib/python3.7/dist-packages/torch/hub.py:267: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  "You are about to download and run code from an untrusted repository. In a future release, this won't "
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
requirements: ipython not found and is required by YOLOv5, attempting auto-update...
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab

  0%|          | 0.00/14.1M [00:00<?, ?B/s]


Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


##TestSet Creation

In [ ]:
start_time=time.time()
vidcap = cv2.VideoCapture('/content/video/footballMatch.mp4')
success,f = vidcap.read()
h,w,_=f.shape
frameSize =(w,h)
count = 0
cv2.waitKey(20)
while success:
    results = yolo(f)    
    A=results.xyxyn[0].data.cpu().numpy() #coordinate of the bounding box 
    B=np.array(f) 
    for i in range(A.shape[0]):
        if (A[i][5])==0.0 or (A[i][5]==32.0): #choose only bounding box with people and ball
            f[int(A[i][1]*(f.shape[0])):int(A[i][3]*(f.shape[0])),int(A[i][0]*(f.shape[1])):int(A[i][2]*(f.shape[1]))]= (0, 0, 0)  
    cv2.imwrite("/content/dataset/full/GT/full/frame%d.png" % count, B)
    B=B-f
    cv2.imwrite("/content/dataset/background/GT/background/frame%d.png" % count, f)
    cv2.imwrite("/content/dataset/foreground/GT/foreground/frame%d.png" % count, B)# save frame as png file      
    success,f = vidcap.read()    
    count += 1
    cv2.waitKey(20)
    if count > 14:
      finish_time=time.time()-start_time
      print('time required to process a frame is '+str(finish_time/15))
      break

time required to process a frame is 1.199316390355428


###frame size reduction

In [ ]:
#reduces the images to 1/4 of their size
def reduceFrameDimension(pathHR,pathLR):
  list_of_files = filter( os.path.isfile,
                        glob.glob(pathHR + '*') )
# Sort list of files based on last modification time in ascending order
  list_of_files = sorted( list_of_files,
                        key = os.path.getmtime)
  quality_val=40
  for image in list_of_files:   
    img = Image.open(image)
    img.thumbnail((img.size[0]/4,img.size[1]/4))
    name = os.path.split(image)  
    file_path = os.path.join(pathLR, name[1])  
    img.save(file_path, quality=quality_val)

In [ ]:
#reduces the background frames to 1/4 of their original size
pathHRBackGround='/content/dataset/background/GT/background/'
pathLRBackGround = '/content/dataset/background/Gaussian/background'
reduceFrameDimension(pathHRBackGround,pathLRBackGround)

#reduces the foreground frames to 1/4 of their original size
pathHRForeground='/content/dataset/foreground/GT/foreground/'
pathLRForeground = '/content/dataset/foreground/Gaussian/foreground'
reduceFrameDimension(pathHRForeground,pathLRForeground)

#reduces the full frames to 1/4 of their original size
pathHRFullFrame='/content/dataset/full/GT/full/'
pathLRFullFrame = '/content/dataset/full/Gaussian/full'
reduceFrameDimension(pathHRFullFrame,pathLRFullFrame)

###create video with background,foreground and full frames

In [ ]:
out = cv2.VideoWriter('/content/video/full.avi',cv2.VideoWriter_fourcc(*'DIVX'), 30, frameSize)

for filename in sorted(glob.glob('/content/dataset/full/GT/full/*.png'), key=os.path.getmtime):
    img = cv2.imread(filename)
    out.write(img)

out.release()



out = cv2.VideoWriter('/content/video/background.avi',cv2.VideoWriter_fourcc(*'DIVX'), 30, frameSize)

for filename in sorted(glob.glob('/content/dataset/background/GT/background/*.png'), key=os.path.getmtime):
    img = cv2.imread(filename)
    out.write(img)

out.release()


out = cv2.VideoWriter('/content/video/foreground.avi',cv2.VideoWriter_fourcc(*'DIVX'), 30, frameSize)

for filename in sorted(glob.glob('/content/dataset/foreground/GT/foreground/*.png'), key=os.path.getmtime):
    img = cv2.imread(filename)
    out.write(img)

out.release()

#EGVSR

import my Mod of EGVSR with all the utilities

In [ ]:
!git clone https://github.com/DamianoGiani/EGVSR.git

Cloning into 'EGVSR'...
remote: Enumerating objects: 564, done.
remote: Counting objects: 100% (313/313), done.
remote: Compressing objects: 100% (202/202), done.
remote: Total 564 (delta 184), reused 186 (delta 109), pack-reused 251
Receiving objects: 100% (564/564), 131.23 MiB | 16.61 MiB/s, done.
Resolving deltas: 100% (234/234), done.


##install the requirements

In [ ]:
!pip3 install -r /content/EGVSR/requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip3 install -r /content/EGVSR/req.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 127 kB 24.2 MB/s 
     |████████████████████████████████| 147 kB 68.9 MB/s 
     |████████████████████████████████| 178 kB 68.9 MB/s 
     |████████████████████████████████| 829 kB 60.0 MB/s 
     |████████████████████████████████| 280 kB 66.9 MB/s 
     |████████████████████████████████| 240 kB 64.1 MB/s 
     |████████████████████████████████| 379 kB 74.3 MB/s 
     |████████████████████████████████| 61 kB 8.8 MB/s 
     |████████████████████████████████| 1.6 MB 58.6 MB/s 
     |████████████████████████████████| 112 kB 77.2 MB/s 
     |████████████████████████████████| 50.4 MB 35 kB/s 
     |████████████████████████████████| 9.9 MB 27.3 MB/s 
     |████████████████████████████████| 67 kB 6.6 MB/s 
     |████████████████████████████████| 283 kB 71.1 MB/s 
     |████████████████████████████████| 227 kB 70.8 MB/s 
     |██████████████████████████████

##Test my Mod of EGVSR and save the resulting frames

In [ ]:
%cd /content/EGVSR

/content/EGVSR


In [ ]:
!sh test.sh BD EGVSR 002

./experiments_BD/EGVSR/002
2022-09-02 15:13:22,912 [INFO]: ========================================
2022-09-02 15:13:22,912 [INFO]: Testing model: EGVSR_iter420000
2022-09-02 15:13:22,912 [INFO]: ========================================
2022-09-02 15:13:26,207 [INFO]: Testing on test1: Background
No metirc need to compute!
torch.Size([15, 540, 960, 3])
No metirc need to compute!
No metirc need to save!
2022-09-02 15:13:54,666 [INFO]: ----------------------------------------
2022-09-02 15:13:54,667 [INFO]: Finish testing
2022-09-02 15:13:54,667 [INFO]: ========================================
--- 31.75466537475586 seconds ---
./experiments_BD/EGVSR/002
2022-09-02 15:13:58,092 [INFO]: ========================================
2022-09-02 15:13:58,092 [INFO]: Testing model: EGVSR_iter420000
2022-09-02 15:13:58,092 [INFO]: ========================================
2022-09-02 15:14:01,661 [INFO]: Testing on test2: Foreground
No metirc need to compute!
torch.Size([15, 540, 960, 3])
No metirc ne

##Create videos of the resulting frames

In [ ]:
out = cv2.VideoWriter('/content/video/resultFirstMethod.avi',cv2.VideoWriter_fourcc(*'DIVX'), 20, frameSize)

for filename in sorted(glob.glob('/content/results/MyFirstMod/EGVSR_iter420000/*.png'), key=os.path.getmtime):
    img = cv2.imread(filename)
    out.write(img)

out.release()

In [ ]:
out = cv2.VideoWriter('/content/video/resultSecondMethod.avi',cv2.VideoWriter_fourcc(*'DIVX'), 20, frameSize)

for filename in sorted(glob.glob('/content/results/MySecondMod/EGVSR_iter420000/*.png'), key=os.path.getmtime):
    img = cv2.imread(filename)
    out.write(img)

out.release()

#Modules Visualization

##Utilities

In [ ]:
class BicubicUpsample(nn.Module):

    def     __init__(self, scale_factor, a=-0.75):
        super(BicubicUpsample, self).__init__()

        # calculate weights
        cubic = torch.FloatTensor([
            [0, a, -2 * a, a],
            [1, 0, -(a + 3), a + 2],
            [0, -a, (2 * a + 3), -(a + 2)],
            [0, 0, a, -a]
        ])  # accord to Eq.(6) in the reference paper

        kernels = [
            torch.matmul(cubic, torch.FloatTensor([1, s, s ** 2, s ** 3]))
            for s in [1.0*d/scale_factor for d in range(scale_factor)]
        ]  # s = x - floor(x)

        # register parameters
        self.scale_factor = scale_factor
        self.register_buffer('kernels', torch.stack(kernels))

    def forward(self, input):
        n, c, h, w = input.size()
        s = self.scale_factor

        # pad input (left, right, top, bottom)
        input = F.pad(input, (1, 2, 1, 2), mode='replicate')

        # calculate output (height)
        kernel_h = self.kernels.repeat(c, 1).view(-1, 1, s, 1)
        output = F.conv2d(input, kernel_h, stride=1, padding=0, groups=c)
        output = output.reshape(
            n, c, s, -1, w + 3).permute(0, 1, 3, 2, 4).reshape(n, c, -1, w + 3)

        # calculate output (width)
        kernel_w = self.kernels.repeat(c, 1).view(-1, 1, 1, s)
        output = F.conv2d(output, kernel_w, stride=1, padding=0, groups=c)
        output = output.reshape(
            n, c, s, h * s, -1).permute(0, 1, 3, 4, 2).reshape(n, c, h * s, -1)

        return output

In [ ]:
class ResidualBlock(nn.Module):
    """ Residual block without batch normalization
    """

    def __init__(self, nf=64):
        super(ResidualBlock, self).__init__()

        self.conv = nn.Sequential(
            nn.Conv2d(nf, nf, 3, 1, 1, bias=True),
            nn.ReLU(inplace=True),
            nn.Conv2d(nf, nf, 3, 1, 1, bias=True))

    def forward(self, x):
        out = self.conv(x) + x

        return

In [ ]:
class BaseSequenceGenerator(nn.Module):
    def __init__(self):
        super(BaseSequenceGenerator, self).__init__()

    def generate_dummy_input(self, lr_size):
        """ use for compute per-step FLOPs and speed
            return random tensors that can be taken as input of <forward>
        """
        return None

    def forward(self, *args, **kwargs):
        """ forward pass for a singe frame
        """
        pass

    def forward_sequence(self, lr_data):
        """ forward pass for a whole sequence (for training)
        """
        pass

    def infer_sequence(self, lr_data, device):
        """ infer for a whole sequence (for inference)
        """
        pass


class BaseSequenceDiscriminator(nn.Module):
    def __init__(self):
        super(BaseSequenceDiscriminator, self).__init__()

    def forward(self, *args, **kwargs):
        """ forward pass for a singe frame
        """
        pass

    def forward_sequence(self, data, args_dict):
        """ forward pass for a whole sequence (for training)
        """
        pass


In [ ]:
def get_upsampling_func(scale=4, degradation='BI'):
    if degradation == 'BI':
        upsample_func = functools.partial(
            F.interpolate, scale_factor=scale, mode='bilinear',
            align_corners=False)

    elif degradation == 'BD':
        upsample_func = BicubicUpsample(scale_factor=scale)

    else:
        raise ValueError('Unrecognized degradation: {}'.format(degradation))

    return upsample_func

##Modules definition

###FNet

In [ ]:
class FNet(nn.Module):
    """ Optical flow estimation network
    """
    
        
    def __init__(self, in_nc):
        super().__init__()
        super(FNet, self).__init__()

        self.encoder1 = nn.Sequential(
            nn.Conv2d(2*in_nc, 32, 3, 1, 1, bias=True),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(32, 32, 3, 1, 1, bias=True),
            nn.LeakyReLU(0.2, inplace=True),
            nn.MaxPool2d(2, 2))

        self.encoder2 = nn.Sequential(
            nn.Conv2d(32, 64, 3, 1, 1, bias=True),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 64, 3, 1, 1, bias=True),
            nn.LeakyReLU(0.2, inplace=True),
            nn.MaxPool2d(2, 2))

        self.encoder3 = nn.Sequential(
            nn.Conv2d(64, 128, 3, 1, 1, bias=True),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(128, 128, 3, 1, 1, bias=True),
            nn.LeakyReLU(0.2, inplace=True),
            nn.MaxPool2d(2, 2))

        self.decoder1 = nn.Sequential(
            nn.Conv2d(128, 256, 3, 1, 1, bias=True),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(256, 256, 3, 1, 1, bias=True),
            nn.LeakyReLU(0.2, inplace=True))

        self.decoder2 = nn.Sequential(
            nn.Conv2d(256, 128, 3, 1, 1, bias=True),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(128, 128, 3, 1, 1, bias=True),
            nn.LeakyReLU(0.2, inplace=True))

        self.decoder3 = nn.Sequential(
            nn.Conv2d(128, 64, 3, 1, 1, bias=True),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 64, 3, 1, 1, bias=True),
            nn.LeakyReLU(0.2, inplace=True))

        self.flow = nn.Sequential(
            nn.Conv2d(64, 32, 3, 1, 1, bias=True),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(32, 2, 3, 1, 1, bias=True))

    def forward(self, x1, x2):
        """ Compute optical flow from x1 to x2
        """

        out = self.encoder1(torch.cat([x1, x2], dim=1))
        out = self.encoder2(out)
        out = self.encoder3(out)
        out = F.interpolate(
            self.decoder1(out), scale_factor=2, mode='bilinear', align_corners=False)
        out = F.interpolate(
            self.decoder2(out), scale_factor=2, mode='bilinear', align_corners=False)
        out = F.interpolate(
            self.decoder3(out), scale_factor=2, mode='bilinear', align_corners=False)
        out = torch.tanh(self.flow(out)) * 24  # 24 is the max velocity

        return out

In [ ]:
fnet=FNet(3)
print(fnet)
with open("fnet.txt", "a") as f:
  print(fnet, file=f)


FNet(
  (encoder1): Sequential(
    (0): Conv2d(6, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): LeakyReLU(negative_slope=0.2, inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (encoder2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): LeakyReLU(negative_slope=0.2, inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (encoder3): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): LeakyReLU(nega

###SRNet

In [ ]:
class SRNet(nn.Module):
    """ Reconstruction & Upsampling network
    """

    def __init__(self, in_nc=3, out_nc=3, nf=64, nb=16, upsample_func=None,
                 scale=4):
        super(SRNet, self).__init__()

        # input conv.
        self.conv_in = nn.Sequential(
            nn.Conv2d((scale**2 + 1) * in_nc, nf, 3, 1, 1, bias=True),
            nn.ReLU(inplace=True))

        # residual blocks
        self.resblocks = nn.Sequential(*[ResidualBlock(nf) for _ in range(nb)])

        # upsampling
        self.conv_up = nn.Sequential(
            nn.ConvTranspose2d(nf, nf, 3, 2, 1, output_padding=1, bias=True),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(nf, nf, 3, 2, 1, output_padding=1, bias=True),
            nn.ReLU(inplace=True))

        self.conv_up_cheap = nn.Sequential(
            nn.PixelShuffle(4),
            nn.ReLU(inplace=True))

        # output conv.
        self.conv_out = nn.Conv2d(4, out_nc, 3, 1, 1, bias=True)

        # upsampling function
        self.upsample_func = upsample_func

    def forward(self, x):
        """ lr_curr: the current lr data in shape nchw
            hr_prev_tran: the previous transformed hr_data in shape n(4*4*c)hw
        """

        out = self.conv_in(x)
        out = self.resblocks(out)
        out = self.conv_up_cheap(out)
        out = self.conv_out(out)
        # out += self.upsample_func(lr_curr)

        return out


In [ ]:
srnet = SRNet()
print(srnet)
with open("srnet.txt", "a") as f:
  print(srnet, file=f)
  

SRNet(
  (conv_in): Sequential(
    (0): Conv2d(51, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
  )
  (resblocks): Sequential(
    (0): ResidualBlock(
      (conv): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU(inplace=True)
        (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )
    )
    (1): ResidualBlock(
      (conv): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU(inplace=True)
        (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )
    )
    (2): ResidualBlock(
      (conv): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU(inplace=True)
        (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )
    )
    (3): ResidualBlock(
      (conv): Sequential(
        (0): Conv2d(64

###FRNet

In [ ]:
class FRNet(BaseSequenceGenerator):
    """ Frame-recurrent network proposed in https://arxiv.org/abs/1801.04590
    """

    def __init__(self, in_nc=3, out_nc=3, nf=64, nb=16, degradation='BI',
                 scale=4):
        super(FRNet, self).__init__()

        self.scale = scale

        # get upsampling function according to the degradation mode
        self.upsample_func = get_upsampling_func(self.scale, degradation)

        # define fnet & srnet
        self.fnet = FNet(in_nc)
        self.srnet = SRNet(in_nc, out_nc, nf, nb, self.upsample_func)

    def generate_dummy_input(self, lr_size):
        c, lr_h, lr_w = lr_size
        s = self.scale

        lr_curr = torch.rand(1, c, lr_h, lr_w, dtype=torch.float32)
        lr_prev = torch.rand(1, c, lr_h, lr_w, dtype=torch.float32)
        hr_prev = torch.rand(1, c, s * lr_h, s * lr_w, dtype=torch.float32)

        data_dict = {
            'lr_curr': lr_curr,
            'lr_prev': lr_prev,
            'hr_prev': hr_prev
        }

        return data_dict

    def forward(self, lr_curr, lr_prev, hr_prev):
        """
            Parameters:
                :param lr_curr: the current lr data in shape nchw
                :param lr_prev: the previous lr data in shape nchw
                :param hr_prev: the previous hr data in shape nc(4h)(4w)
        """

        # estimate lr flow (lr_curr -> lr_prev)
        lr_flow = self.fnet(lr_curr, lr_prev)

        # pad if size is not a multiple of 8
        pad_h = lr_curr.size(2) - lr_curr.size(2) // 8 * 8
        pad_w = lr_curr.size(3) - lr_curr.size(3) // 8 * 8
        lr_flow_pad = F.pad(lr_flow, (0, pad_w, 0, pad_h), 'reflect')

        # upsample lr flow
        hr_flow = self.scale * self.upsample_func(lr_flow_pad)

        # warp hr_prev
        hr_prev_warp = backward_warp(hr_prev, hr_flow)

        # compute hr_curr
        hr_curr = self.srnet(lr_curr, space_to_depth(hr_prev_warp, self.scale))
        concres= torch.cat([lr_curr, space_to_depth(hr_prev_warp)],1)
        return hr_curr,concres

    def forward_sequence(self, lr_data):
        """
            Parameters:
                :param lr_data: lr data in shape ntchw
        """

        n, t, c, lr_h, lr_w = lr_data.size()
        hr_h, hr_w = lr_h * self.scale, lr_w * self.scale

        # calculate optical flows
        lr_prev = lr_data[:, :-1, ...].reshape(n * (t - 1), c, lr_h, lr_w)
        lr_curr = lr_data[:, 1:, ...].reshape(n * (t - 1), c, lr_h, lr_w)
        file_name = "sample.pkl"

        open_file = open(file_name, "wb")
        pickle.dump(lr_curr, open_file)
        open_file.close()
        lr_flow = self.fnet(lr_curr, lr_prev)  # n*(t-1),2,h,w

        # upsample lr flows
        hr_flow = self.scale * self.upsample_func(lr_flow)
        hr_flow = hr_flow.view(n, (t - 1), 2, hr_h, hr_w)

        # compute the first hr data
        hr_data = []
        hr_prev = self.srnet(
            lr_data[:, 0, ...],
            torch.zeros(n, (self.scale**2)*c, lr_h, lr_w, dtype=torch.float32,
                        device=lr_data.device))
        hr_data.append(hr_prev)

        # compute the remaining hr data
        for i in range(1, t):
            # warp hr_prev
            hr_prev_warp = backward_warp(hr_prev, hr_flow[:, i - 1, ...])

            # compute hr_curr
            hr_curr = self.srnet(
                lr_data[:, i, ...],
                space_to_depth(hr_prev_warp, self.scale))

            # save and update
            hr_data.append(hr_curr)
            hr_prev = hr_curr

        hr_data = torch.stack(hr_data, dim=1)  # n,t,c,hr_h,hr_w

        # construct output dict
        ret_dict = {
            'hr_data': hr_data,  # n,t,c,hr_h,hr_w
            'hr_flow': hr_flow,  # n,t,2,hr_h,hr_w
            'lr_prev': lr_prev,  # n(t-1),c,lr_h,lr_w
            'lr_curr': lr_curr,  # n(t-1),c,lr_h,lr_w
            'lr_flow': lr_flow,  # n(t-1),2,lr_h,lr_w
        }

        return ret_dict

    def infer_sequence(self, lr_data, device):
        """
            Parameters:
                :param lr_data: torch.FloatTensor in shape tchw
                :param device: torch.device

                :return hr_seq: uint8 np.ndarray in shape tchw
        """

        # setup params
        tot_frm, c, h, w = lr_data.size()
        s = self.scale

        # forward
        hr_seq = []
        lr_prev = torch.zeros(1, c, h, w, dtype=torch.float32).to(device)
        hr_prev = torch.zeros(
            1, c, s * h, s * w, dtype=torch.float32).to(device)
        tensorlist = []
        for i in range(tot_frm):
            with torch.no_grad():
                self.eval()

                lr_curr = lr_data[i: i + 1, ...].to(device)
                hr_curr,concres = self.forward(lr_curr, lr_prev, hr_prev) ##la domanda è cosa fare con questo concres? che è un tensore attualmente
                lr_prev, hr_prev = lr_curr, hr_curr
                tensorlist.append(concres) #######non so se devo prima fare queeze in numpy
                hr_frm = hr_curr.squeeze(0).cpu().numpy()  # chw|rgb|uint8

            hr_seq.append(float32_to_uint8(hr_frm))
        file_name = "sample.pkl"

        open_file = open(file_name, "wb")
        pickle.dump(tensorlist, open_file)
        open_file.close()

        return np.stack(hr_seq).transpose(0, 2, 3, 1)  # thwc


In [ ]:
frnet=FRNet()
print(frnet)
with open("frnet.txt", "a") as f:
  print(frnet, file=f)


FRNet(
  (fnet): FNet(
    (encoder1): Sequential(
      (0): Conv2d(6, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): LeakyReLU(negative_slope=0.2, inplace=True)
      (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): LeakyReLU(negative_slope=0.2, inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (encoder2): Sequential(
      (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): LeakyReLU(negative_slope=0.2, inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): LeakyReLU(negative_slope=0.2, inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (encoder3): Sequential(
      (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): LeakyReLU(negative_slope=0.2, inplace=True)
      (2): Conv2d(128, 128, kernel_size=(3, 3), s

#Result

In [ ]:
%cd /content

/content


clonation of the code for PerceptualSimilarity measurements

In [ ]:
!git clone https://github.com/richzhang/PerceptualSimilarity


Cloning into 'PerceptualSimilarity'...
remote: Enumerating objects: 778, done.
remote: Counting objects: 100% (775/775), done.
remote: Compressing objects: 100% (316/316), done.
remote: Total 778 (delta 457), reused 774 (delta 456), pack-reused 3
Receiving objects: 100% (778/778), 8.75 MiB | 7.37 MiB/s, done.
Resolving deltas: 100% (457/457), done.


##Methods definition of the metrics

In [ ]:
%cd PerceptualSimilarity
import lpips

/content/PerceptualSimilarity


In [ ]:
def PSNR(path1,path2,mod=''):
  PSNR=[]
  PSNR1=[]
  if mod=='baseline':    
    list_of_files = filter( os.path.isfile,
                        glob.glob(path1 + '*'))
    list_of_files = sorted( list_of_files,
                        key = os.path.getmtime)
    list_of_files2 = filter( os.path.isfile,
                        glob.glob(path2 + '*'))
    list_of_files2 = sorted(list_of_files2,key = os.path.getmtime)   
    j=0
    for image in sorted( list_of_files,key = os.path.getmtime):      
      f = cv2.imread(image)
      g = cv2.imread(list_of_files2[j], 1)
      psnr1= cv2.PSNR(f, g)
      PSNR1.append(psnr1)
      results = yolo(f)    
      A=results.xyxyn[0].data.cpu().numpy()
      j+=1    
      for i in range(A.shape[0]):
          c=np.array(f[int(A[i][1]*(f.shape[0])):int(A[i][3]*(f.shape[0])),int(A[i][0]*(f.shape[1])):int(A[i][2]*(f.shape[1]))])
          d=np.array(g[int(A[i][1]*(f.shape[0])):int(A[i][3]*(f.shape[0])),int(A[i][0]*(f.shape[1])):int(A[i][2]*(f.shape[1]))]) 
          psnr= cv2.PSNR(c, d)
          PSNR.append(psnr)
  else:
    images1 = [file for file in os.listdir(path1) if file.endswith(('png'))]  
    images2 = [file for file in os.listdir(path2) if file.endswith(('png'))]
    
    for i in range(len(images1)):       
        f = cv2.imread(path1+'/'+images1[i])
        g = cv2.imread(path2+'/'+images2[i], 1)
        psnr1= cv2.PSNR(f, g)
        PSNR1.append(psnr1)
        results = yolo(f)    
        A=results.xyxyn[0].data.cpu().numpy()    
        for i in range(A.shape[0]):
            c=np.array(f[int(A[i][1]*(f.shape[0])):int(A[i][3]*(f.shape[0])),int(A[i][0]*(f.shape[1])):int(A[i][2]*(f.shape[1]))])
            d=np.array(g[int(A[i][1]*(f.shape[0])):int(A[i][3]*(f.shape[0])),int(A[i][0]*(f.shape[1])):int(A[i][2]*(f.shape[1]))]) 
            psnr= cv2.PSNR(c, d)
            PSNR.append(psnr)     
  psnrpatch=np.sum(PSNR)/len(PSNR)
  print("PSNR calcolato su patch è: " + str(psnrpatch))
  psnrfull=np.sum(PSNR1)/len(PSNR1)  
  print("PSNR calcolato intera immagine è: " + str(psnrfull));
  return psnr,psnrfull

In [ ]:
def mse(imageA, imageB):
 # the 'Mean Squared Error' between the two images is the sum of the squared difference between the two images
 mse_error = np.sum((imageA.astype("float") - imageB.astype("float")) ** 2)
 mse_error /= float(imageA.shape[0] * imageA.shape[1])
	
 # return the MSE. The lower the error, the more "similar" the two images are.
 return mse_error

def compare(imageA, imageB):
 # Calculate the MSE and SSIM
 m = mse(imageA, imageB)
 s = ssim(imageA, imageB)
 return s

In [ ]:
def SSIM(path1,path2,mod=''):  
  SSIM=[]
  MSE=[]
  SSIMPatch=[]
  MSEPatch=[]
  PSNR=[]
  PSNR1=[]
  if mod=='baseline':    
    list_of_files = filter( os.path.isfile,
                        glob.glob(path1 + '*'))
    list_of_files = sorted( list_of_files,
                        key = os.path.getmtime)
    list_of_files2 = filter( os.path.isfile,
                        glob.glob(path2 + '*'))
    list_of_files2 = sorted(list_of_files2,key = os.path.getmtime)   
    j=0
    for image in sorted( list_of_files,key = os.path.getmtime): 
        f = cv2.imread(image)
        g = cv2.imread(list_of_files2[j], 1)
        gray1 = cv2.cvtColor(f, cv2.COLOR_BGR2GRAY)
        gray2 = cv2.cvtColor(g, cv2.COLOR_BGR2GRAY)    
        mse_value = mse(gray1, gray2)
        ssim_value = compare(gray1, gray2)
        SSIM.append(ssim_value)
        MSE.append(mse_value)
        results = yolo(f)    
        A=results.xyxyn[0].data.cpu().numpy()
        j+=1         
        for i in range(A.shape[0]):
          c=np.array(gray1[int(A[i][1]*(f.shape[0])):int(A[i][3]*(f.shape[0])),int(A[i][0]*(f.shape[1])):int(A[i][2]*(f.shape[1]))])
          d=np.array(gray2[int(A[i][1]*(f.shape[0])):int(A[i][3]*(f.shape[0])),int(A[i][0]*(f.shape[1])):int(A[i][2]*(f.shape[1]))]) 
          mse_valuepatch = mse(c, d)
          ssim_valuepatch = compare(c, d)
          SSIMPatch.append(ssim_valuepatch)
          MSEPatch.append(mse_valuepatch)
  
  else:
    images1 = [file for file in os.listdir(path1) if file.endswith(('png'))]
    images2 = [file for file in os.listdir(path2) if file.endswith(('png'))]
    for i in range(len(images1)):
        f = cv2.imread(path1+'/'+images1[i])
        g = cv2.imread(path2+'/'+images2[i], 1)
        gray1 = cv2.cvtColor(f, cv2.COLOR_BGR2GRAY)
        gray2 = cv2.cvtColor(g, cv2.COLOR_BGR2GRAY)    
        mse_value = mse(gray1, gray2)
        ssim_value = compare(gray1, gray2)
        SSIM.append(ssim_value)
        MSE.append(mse_value)
        results = yolo(f)    
        A=results.xyxyn[0].data.cpu().numpy()         
        for i in range(A.shape[0]):
          c=np.array(gray1[int(A[i][1]*(f.shape[0])):int(A[i][3]*(f.shape[0])),int(A[i][0]*(f.shape[1])):int(A[i][2]*(f.shape[1]))])
          d=np.array(gray2[int(A[i][1]*(f.shape[0])):int(A[i][3]*(f.shape[0])),int(A[i][0]*(f.shape[1])):int(A[i][2]*(f.shape[1]))]) 
          mse_valuepatch = mse(c, d)
          ssim_valuepatch = compare(c, d)
          SSIMPatch.append(ssim_valuepatch)
          MSEPatch.append(mse_valuepatch)
  
  ssimtot=np.sum(SSIM)/len(SSIM)
  print("SSIM calcolato su intere immagini è: " + str(ssimtot))  
   
  msetot=np.sum(MSE)/len(MSE)
  #print("MSE calcolato su intere immagini è: " + str(msetot))
  
  ssimtotpatch=np.sum(SSIMPatch)/len(SSIMPatch)
  print("SSIM calcolato su Patch è: " + str(ssimtotpatch));         
  msetotpatch=np.sum(MSEPatch)/len(MSEPatch)
  #print("MSE calcolato su Patch è: " + str(msetotpatch));   
  return ssimtot,msetot,ssimtotpatch,msetotpatch

In [ ]:
def DIST(path1,path2,mod=''):  
  
  DIST=[]
  DISTPatch=[]
  loss_fn = lpips.LPIPS(net='alex')
  loss_fn.cuda()

  if mod=='baseline':    
    list_of_files = filter( os.path.isfile,
                        glob.glob(path1 + '*'))
    list_of_files = sorted( list_of_files,
                        key = os.path.getmtime)
    list_of_files2 = filter( os.path.isfile,
                        glob.glob(path2 + '*'))
    list_of_files2 = sorted(list_of_files2,key = os.path.getmtime)   
    j=0
    for image in sorted( list_of_files,key = os.path.getmtime):
      f = cv2.imread(image)
      g = cv2.imread(list_of_files2[j], 1)
      results = yolo(f)    
      A=results.xyxyn[0].data.cpu().numpy()
      j+=1 
      for i in range(A.shape[0]-1):
          c=np.array(f[int(A[i][1]*(f.shape[0])):int(A[i][3]*(f.shape[0])),int(A[i][0]*(f.shape[1])):int(A[i][2]*(f.shape[1]))])
          d=np.array(g[int(A[i][1]*(f.shape[0])):int(A[i][3]*(f.shape[0])),int(A[i][0]*(f.shape[1])):int(A[i][2]*(f.shape[1]))])
          
          h=(c / 255./2. - 1)[:, :, :, np.newaxis].transpose((3, 2, 0, 1))
          hh=(d / 255./2. - 1)[:, :, :, np.newaxis].transpose((3, 2, 0, 1))
          h=torch.Tensor(h)
          hh=torch.Tensor(hh)
          h = h.cuda()
          hh = hh.cuda()
          dist02 = loss_fn.forward(h,hh)
          DISTPatch.append(dist02[0][0][0][0].cpu().detach().numpy())
          torch.cuda.empty_cache()  
     
      k=(f / 255./2. - 1)[:, :, :, np.newaxis].transpose((3, 2, 0, 1))
      kk=(g / 255./2. - 1)[:, :, :, np.newaxis].transpose((3, 2, 0, 1))
      k=torch.Tensor(k)
      kk=torch.Tensor(kk)
      k = k.cuda()
      kk = kk.cuda()
      dist01 = loss_fn.forward(k,kk)
      DIST.append(dist01[0][0][0][0].cpu().detach().numpy())
      torch.cuda.empty_cache()
  else:    

    images1 = [file for file in os.listdir(path1) if file.endswith(('png'))]  
    images2 = [file for file in os.listdir(path2) if file.endswith(('png'))]
    for i in range(len(images1)):
        f = cv2.imread(path1+'/'+images1[i])
        g = cv2.imread(path2+'/'+images2[i], 1)
        results = yolo(f)    
        A=results.xyxyn[0].data.cpu().numpy()
        for i in range(A.shape[0]-1):
            c=np.array(f[int(A[i][1]*(f.shape[0])):int(A[i][3]*(f.shape[0])),int(A[i][0]*(f.shape[1])):int(A[i][2]*(f.shape[1]))])
            d=np.array(g[int(A[i][1]*(f.shape[0])):int(A[i][3]*(f.shape[0])),int(A[i][0]*(f.shape[1])):int(A[i][2]*(f.shape[1]))])
            
            h=(c / 255./2. - 1)[:, :, :, np.newaxis].transpose((3, 2, 0, 1))
            hh=(d / 255./2. - 1)[:, :, :, np.newaxis].transpose((3, 2, 0, 1))
            h=torch.Tensor(h)
            hh=torch.Tensor(hh)
            h = h.cuda()
            hh = hh.cuda()
            dist02 = loss_fn.forward(h,hh)
            DISTPatch.append(dist02[0][0][0][0].cpu().detach().numpy())
            torch.cuda.empty_cache()          
        k=(f / 255./2. - 1)[:, :, :, np.newaxis].transpose((3, 2, 0, 1))
        kk=(g / 255./2. - 1)[:, :, :, np.newaxis].transpose((3, 2, 0, 1))
        k=torch.Tensor(k)
        kk=torch.Tensor(kk)
        k = k.cuda()
        kk = kk.cuda()
        dist01 = loss_fn.forward(k,kk)
        DIST.append(dist01[0][0][0][0].cpu().detach().numpy())
        torch.cuda.empty_cache()
        
  dist1=np.sum(DIST)/len(DIST)  
  print("Distance su tutta immagine : " + str(dist1))
  distpetch=np.sum(DISTPatch)/len(DISTPatch)  
  print("Distance su patch : " + str(distpetch))
  return dist1 ,distpetch 

##Call methods

###Results First Method

In [ ]:
PSNRBaselinepatch,baselineFull=PSNR('/content/dataset/full/GT/full/','/content/results/Full/EGVSR_iter420000/full/',mod='baseline')
PSNRMyMod,MyModFull=PSNR('/content/dataset/full/GT/full','/content/results/MyFirstMod/EGVSR_iter420000')

PSNR calcolato su patch è: 25.192784819883457
PSNR calcolato intera immagine è: 29.60457198583706
PSNR calcolato su patch è: 15.49131223154234
PSNR calcolato intera immagine è: 16.72538412519588


In [ ]:
SSIMBaselineFull,MSEbaselineFull,SSIMBaselinepatch,MSEbaselinepatch=SSIM('/content/dataset/full/GT/full/','/content/results/Full/EGVSR_iter420000/full/',mod='baseline')
SSIMMymodFull,MSEmymodFull,SSIMMyModpatch,MSEMuymodpatch=SSIM('/content/dataset/full/GT/full','/content/results/MyFirstMod/EGVSR_iter420000')

SSIM calcolato su intere immagini è: 0.9398859157592359
SSIM calcolato su Patch è: 0.8416278529907878
SSIM calcolato su intere immagini è: 0.21731713288380705
SSIM calcolato su Patch è: 0.2685577390066006


In [ ]:
DISTBaselineFull,DISTBaselinePatch=DIST('/content/dataset/full/GT/full/','/content/results/Full/EGVSR_iter420000/full/',mod='baseline')
DISTMymodFull,DISTMymodPatch=DIST('/content/dataset/full/GT/full','/content/results/MyFirstMod/EGVSR_iter420000')

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth


  0%|          | 0.00/233M [00:00<?, ?B/s]

Loading model from: /content/PerceptualSimilarity/lpips/weights/v0.1/alex.pth
Distance su tutta immagine : 0.0162030299504598
Distance su patch : 0.03977130232630549
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.


Loading model from: /content/PerceptualSimilarity/lpips/weights/v0.1/alex.pth
Distance su tutta immagine : 0.43123111724853513
Distance su patch : 0.38922186155576965


###Results Second Method

In [ ]:
PSNRBaselinepatch,baselineFull=PSNR('/content/dataset/full/GT/full/','/content/results/Full/EGVSR_iter420000/full/',mod='baseline')
PSNRMyMod,MyModFull=PSNR('/content/dataset/full/GT/full','/content/results/MySecondMod/EGVSR_iter420000')

PSNR calcolato su patch è: 25.192784819883457
PSNR calcolato intera immagine è: 29.60457198583706
PSNR calcolato su patch è: 21.83275839182907
PSNR calcolato intera immagine è: 28.578568766683116


In [ ]:
SSIMBaselineFull,MSEbaselineFull,SSIMBaselinepatch,MSEbaselinepatch=SSIM('/content/dataset/full/GT/full/','/content/results/Full/EGVSR_iter420000/full/',mod='baseline')
SSIMMymodFull,MSEmymodFull,SSIMMyModpatch,MSEMuymodpatch=SSIM('/content/dataset/full/GT/full','/content/results/MySecondMod/EGVSR_iter420000')

SSIM calcolato su intere immagini è: 0.9398859157592359
SSIM calcolato su Patch è: 0.8416278529907878
SSIM calcolato su intere immagini è: 0.9312943967737575
SSIM calcolato su Patch è: 0.7740869733454896


In [ ]:
DISTBaselineFull,DISTBaselinePatch=DIST('/content/dataset/full/GT/full/','/content/results/Full/EGVSR_iter420000/full/',mod='baseline')
DISTMymodFull,DISTMymodPatch=DIST('/content/dataset/full/GT/full','/content/results/MySecondMod/EGVSR_iter420000')

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.


Loading model from: /content/PerceptualSimilarity/lpips/weights/v0.1/alex.pth
Distance su tutta immagine : 0.0162030299504598
Distance su patch : 0.03977130232630549
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.


Loading model from: /content/PerceptualSimilarity/lpips/weights/v0.1/alex.pth
Distance su tutta immagine : 0.04174271821975708
Distance su patch : 0.15467400164217562
